In [1]:
from bs4 import BeautifulSoup
import requests, re, time

In [93]:
# vergadering = soup.find(id='broodtekst')

In [14]:
achternamen = []
partijen = []
teksten = []

In [15]:
def parse_item(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    for spreekbeurt in soup.find_all(class_=['spreekbeurt', 'spreekbeurt eerst']):
        spreker = spreekbeurt.find(class_='spreker')
        achternaam = spreker.find(class_='achternaam').get_text()
        partij = spreker.find(class_='politiek')
        if not partij is None:
            partij = partij.get_text()

        tekst = spreekbeurt.find_all(class_=['alineagroep', 'motie'])
        tekst = ' '.join([alinea.get_text() for alinea in tekst])

        achternamen.append(achternaam)
        partijen.append(partij)
        teksten.append(tekst)

In [16]:
year = '2013-2014'

In [26]:
for i in range(1,1000): # !
    print(i)
    url = 'https://zoek.officielebekendmakingen.nl/rss/handelingen/TK/'+year+'/'+str(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    for br in soup.find_all('br'):
        br.replace_with('\n')
    urls = re.findall('<link/>(.*?)<category>', str(soup))
    if len(urls)==0:
        break
    for url in urls[1:]:
        parse_item(url)
    time.sleep(1)

49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109


In [18]:
len(teksten)

29162

In [27]:
woorden = 0
for t in teksten:
    woorden += len(t)
print('Woorden: {}'.format(woorden))

Woorden: 43396964


In [28]:
import pandas as pd

In [29]:
df = pd.DataFrame([achternamen, partijen, teksten], index=['Naam', 'Partij', 'Uitspraak']).T

In [30]:
df.head()

,Naam,Partij,Uitspraak
0,voorzitter,None,\nIk deel aan de Kamer mee dat het volgende li...
1,voorzitter,None,\nIk geef het woord aan de minister van Financ...
2,Dijsselbloem,None,\nMevrouw de voorzitter. Het is een eer om hie...
3,voorzitter,None,\nIk zal er zuinig op zijn.\n \nIk dank de min...
4,voorzitter,None,"\nIk stel voor, als commissies bedoeld in arti..."


In [31]:
year

'2013-2014'

In [32]:
df.to_csv('uitspraken Tweede Kamer'+year+'.csv', index=False)

<hr>

# Scrape meest recente 2 maanden aan plenaire vergaderingen Tweede Kamer

In [2]:
deelnemers = set()
deelnemers.add('de voorzitter')

In [3]:
uitspraken = {}

In [4]:
pattern = re.compile('(\n\n)')

In [5]:
for i in range(74, 98):
    print(i)
    url = 'https://www.tweedekamer.nl/kamerstukken/plenaire_verslagen/detail?vj=2016-2017&nr='+str(i)+'&version=2'
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, 'lxml')
    vergadering = soup.find_all(id="vergadering")[0]
    
    for br in vergadering.find_all('br'):
        br.replace_with('\n')
    
    # verzamel lijst van deelnemers
    for naam in vergadering.find_all(class_='naam'):
        t = naam.get_text(strip=True).lower()
        if not t[:7] == 'termijn' and not t[:5] == 'motie' and not t[:8] == 'stemming':
            deelnemers.add(t)
        
    for d in deelnemers:
        if not d in uitspraken.keys():
            uitspraken[d] = []
        
    text = vergadering.get_text()
    
    # verzamel uitspraken per deelnemer
    for el in re.split(pattern, text):
        el = el.lower().splitlines()
        if len(el) < 3:
            continue
        if el[1][:-1] in deelnemers:
            uitspraak = ' '.join([d.strip() for d in el[2:]])
            uitspraken[el[1][:-1]].append(uitspraak)
            
    time.sleep(10)

74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [9]:
totaal = 0
woorden = 0
for deelnemer in deelnemers:
    totaal += len(uitspraken[deelnemer])
    print(len(uitspraken[deelnemer]), deelnemer)
    for uitspraak in uitspraken[deelnemer]:
        woorden += len(uitspraak)
print()
print('Totaal aantal uitspraken: {}'.format(totaal))
print('Totaal aantal woorden: {}'.format(woorden))

0 vragen futselaar
20 mevrouw van engelshoven (d66)
0 vragen arno rutte
8 mevrouw yeşilgöz-zegerius (vvd)
29 de heer de jong (pvv)
51 de heer markuszower (pvv)
45 de heer verhoeven (d66)
56 staatssecretaris klijnsma
102 minister plasterk
9 de heer koopmans (vvd)
10 de heer azmani (vvd)
80 de heer dijkgraaf (sgp)
0 vragen özdil
12 minister koenders
30 de heer krol (50plus)
74 mevrouw dik-faber (christenunie)
58 de heer van raak (sp)
17 mevrouw belhaj (d66)
13 mevrouw van den hul (pvda)
44 de heer kuzu (denk)
34 mevrouw becker (vvd)
0 lijst van controversiële onderwerpen
12 mevrouw van den berg (cda)
8 mevrouw hermans (vvd)
56 minister schultz van haegen-maas geesteranus
22 de heer martin bosma (pvv)
74 mevrouw van toorenburg (cda)
48 de heer futselaar (sp)
8 mevrouw pia dijkstra (d66)
22 de heer rog (cda)
77 minister bussemaker
179 de heer omtzigt (cda)
76 mevrouw kröger (groenlinks)
181 minister dijsselbloem
0 vragen groothuizen
4 de heer sjoerdsma (d66)
36 de heer segers (christenunie

In [8]:
import json
with open('uitspraken.json', 'w') as outfile:  
    json.dump(uitspraken, outfile, indent=4)

<hr>

In [ ]:
import re
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      text )  # The text to search

In [ ]:
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

In [ ]:
from nltk.corpus import stopwords

In [ ]:
words = [w for w in words if not w in stopwords.words("dutch")]
clean_text = ' '.join(words)

In [ ]:
clean_text

In [ ]:
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()